In [1]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from transformers import AutoModel, AutoProcessor
import warnings
warnings.filterwarnings('ignore')

2024-10-01 13:08:48.601405: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-01 13:08:48.609955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 13:08:48.619539: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 13:08:48.622630: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-01 13:08:48.630169: I tensorflow/core/platform/cpu_feature_guar

In [2]:
df = pd.read_csv('myntradataset/styles.csv', usecols=['id',  'subCategory', 'articleType'])
print(f"Style {type(df)} {df.shape}")
print(df.head(10))

Style <class 'pandas.core.frame.DataFrame'> (44446, 3)
      id subCategory  articleType
0  15970     Topwear       Shirts
1  39386  Bottomwear        Jeans
2  59263     Watches      Watches
3  21379  Bottomwear  Track Pants
4  53759     Topwear      Tshirts
5   1855     Topwear      Tshirts
6  30805     Topwear       Shirts
7  26960     Topwear       Shirts
8  29114       Socks        Socks
9  30039     Watches      Watches


In [3]:
# image = [Image.open("outputs/t-shirt.jpeg")]
image = [Image.open("outputs/flipflops.jpg")]
# text = ["hat", "t-shirt", "shoes", 'gloves', 'headwear']
text = list(set(
    list(df["articleType"].unique()) 
    + list(df["subCategory"].unique())
))

In [5]:

model = AutoModel.from_pretrained('Marqo/marqo-fashionCLIP', trust_remote_code=True)
processor = AutoProcessor.from_pretrained('Marqo/marqo-fashionCLIP', trust_remote_code=True)
processed = processor(text=text, images=image, padding='max_length', return_tensors="pt")

with torch.no_grad():
    image_features = model.get_image_features(processed['pixel_values'], normalize=True)
    text_features = model.get_text_features(processed['input_ids'], normalize=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print(f"Label probs: {text_probs.shape}:\n{text_probs}")
top_result = np.argmax(text_probs).item()
print(top_result, text[top_result])

You are using a model of type clip to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


Label probs: torch.Size([1, 172]):
tensor([[1.3774e-10, 2.4650e-10, 6.9929e-12, 1.3150e-11, 2.7682e-11, 3.7650e-11,
         7.4156e-12, 2.7348e-08, 1.2045e-08, 3.2158e-08, 5.0688e-10, 3.2880e-10,
         3.1383e-10, 7.3545e-13, 3.4146e-09, 4.8336e-10, 7.8633e-12, 1.3050e-11,
         8.2175e-11, 1.1065e-09, 1.9445e-10, 6.2800e-10, 1.5727e-11, 3.4921e-10,
         7.2185e-12, 5.1403e-12, 2.2712e-10, 1.5978e-11, 5.1202e-09, 7.2880e-06,
         1.0848e-09, 6.3632e-02, 5.5984e-07, 1.9029e-11, 2.5247e-09, 6.9492e-08,
         2.9283e-13, 3.2953e-06, 1.8576e-11, 1.1767e-10, 2.6771e-08, 6.8365e-10,
         2.9132e-10, 1.7664e-07, 3.1701e-09, 1.1281e-06, 1.8886e-08, 1.1347e-08,
         2.0812e-09, 2.3805e-11, 1.4380e-08, 3.6989e-11, 4.6018e-08, 1.1819e-10,
         8.8848e-09, 1.4434e-09, 4.2408e-09, 2.9698e-12, 8.5448e-10, 8.6446e-11,
         8.7564e-11, 2.0664e-11, 8.9143e-01, 5.3104e-10, 1.8221e-09, 7.8513e-09,
         8.4148e-10, 2.3863e-07, 1.9958e-12, 5.1662e-09, 3.2063e-13, 3.463